In [ ]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicioProcessamento = time.time()

In [ ]:
!pip install -q -U pip setuptools wheel
!pip install -q torch transformers numpy nltk

In [ ]:
import torch
from transformers import BertTokenizer,BertForTokenClassification, BertConfig
import numpy as np
import json

In [ ]:
#biblioteca de logging
import logging

#Formato da mensagem
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempoArredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempoArredondado))    

In [ ]:
MODELS_DIR = [
              r"pucpr/clinicalnerpt-chemical"
              ,r"pucpr/clinicalnerpt-diagnostic"
              ,r"pucpr/clinicalnerpt-disease"             
              ,r"pucpr/clinicalnerpt-disorder"             
              ,r"pucpr/clinicalnerpt-finding"             
              ,r"pucpr/clinicalnerpt-healthcare"             
              ,r"pucpr/clinicalnerpt-laboratory"             
              ,r"pucpr/clinicalnerpt-medical"             
              ,r"pucpr/clinicalnerpt-pharmacologic"             
              ,r"pucpr/clinicalnerpt-procedure"             
              ,r"pucpr/clinicalnerpt-quantitative"             
              ,r"pucpr/clinicalnerpt-sign"             
              ,r"pucpr/clinicalnerpt-therapeutic"             
]

In [ ]:
def predictBERTNER(sentencas, MODEL_DIR):
        
    model = BertForTokenClassification.from_pretrained(MODEL_DIR)
    tokenizer = BertTokenizer.from_pretrained(MODEL_DIR, do_lower_case=True)

    config = BertConfig.from_pretrained(MODEL_DIR)
    print(config.id2label)
    print(type(config.id2label))
    dic_label=config.id2label

    # with open(MODEL_DIR + '/idx2tag.json', 'r') as filehandle:
    #     idx2tag = json.load(filehandle)
    #     logging.info(idx2tag)

    # idx2tag = {"0": "O",
    #        "1": "B-MET", 
    #        "2": "I-MET", 
    #        "3": "B-DAT", 
    #        "4": "tag", 
    #        "5": "I-DAT", 
    #        "6": "PAD"} 
        
    predictedModel=[]
    
    for test_sentence in sentencas:
        tokenized_sentence = tokenizer.encode(test_sentence)
        input_ids = torch.tensor([tokenized_sentence])#.cuda()
        
        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        
        # join bpe split tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(label_idx)
                new_tokens.append(token)
            
        FinalLabelSentence = []
        for token, label in zip(new_tokens, new_labels):
            label = dic_label.get(label)
            if label == "O" or label == "X":
                FinalLabelSentence.append("O")
            else:
                FinalLabelSentence.append(label)
                
        predictedModel.append(FinalLabelSentence[1:-1]) # delete [SEP] and [CLS]
        
            
    return predictedModel


In [ ]:
import nltk  
nltk.download('punkt')  
from nltk import tokenize    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# THE MODEL ACCEPTS ONLY LOWER
test_sentence1 = "Paciente com Sepse pulmonar em D8 tazocin (pciente não recebeu por 2 dias Atb).".lower()
test_sentence2 = "Acesso venoso central em subclavia D duplolumen recebendo solução salina e glicosada em BI.".lower()

# test_sentence3 = "Solicito RT-PCR SARSCOV 2. Forneço atestado médico 10 dias. Isolamento social Oriento sinais de alarme e de necessidade novo teleatendimento ou de atendimento presencial.".lower()
# test_sentence4 = "Orientações gerais, repouso, alimentação e hidratação Prescrevo sintomáticos Oriento monitorar SpO2.".lower()

test_sentence3 = "Prescrevo Cetoprofeno e descolonização da pele com Triclosan, e manter Mometasona;"
test_sentence4 = "Oriento buscar atendimento com Dermatologista caso não haja melhora;"

test_sentence_tokenized = [tokenize.word_tokenize(test_sentence1, language='portuguese'),
                           tokenize.word_tokenize(test_sentence2, language='portuguese'),
                           tokenize.word_tokenize(test_sentence3, language='portuguese'),
                           tokenize.word_tokenize(test_sentence4, language='portuguese')
                           ] 
logging.info(test_sentence_tokenized)

2022-05-27 12:05:31,685 : INFO : [['paciente', 'com', 'sepse', 'pulmonar', 'em', 'd8', 'tazocin', '(', 'pciente', 'não', 'recebeu', 'por', '2', 'dias', 'atb', ')', '.'], ['acesso', 'venoso', 'central', 'em', 'subclavia', 'd', 'duplolumen', 'recebendo', 'solução', 'salina', 'e', 'glicosada', 'em', 'bi', '.'], ['Prescrevo', 'Cetoprofeno', 'e', 'descolonização', 'da', 'pele', 'com', 'Triclosan', ',', 'e', 'manter', 'Mometasona', ';'], ['Oriento', 'buscar', 'atendimento', 'com', 'Dermatologista', 'caso', 'não', 'haja', 'melhora', ';']]


In [ ]:

# tags = predictBERTNER(test_sentence_tokenized)
# tags

In [ ]:
predictedModels = []
for MODEL_DIR in MODELS_DIR:

        try:
            ner_results = predictBERTNER(test_sentence_tokenized, MODEL_DIR)
            predictedModels.append(ner_results)

        except:
            print('Erro BERT (frase muito grande provavelmente)')
            raise
logging.info(predictedModels)

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/676M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/151 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/151 [00:00<?, ?B/s]

{0: 'O', 1: 'B-TherapeuticPreventiveProcedure', 2: 'I-TherapeuticPreventiveProcedure'}
<class 'dict'>


2022-05-27 12:14:24,254 : INFO : [[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']], [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']], [['O', 'O', 'B-DiseaseOrSyndrome', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']], [['O', 'O', 'B-Disorder', 'B-Disorder', 'I-Disorder', 'I-Disorder', 'I-Disorder', 'O', 'O', '

In [ ]:
result = ''
temLabel = False

for i, sentenca in enumerate(test_sentence_tokenized):
  if len(sentenca)<1:
    continue

  for j, token in enumerate(sentenca):
    temLabel = False
    result = result+'\''+token.strip()+'\':'+'{'
    # print('"',token.strip(),'":','{')
    for k, MODEL_DIR in enumerate(MODELS_DIR):
      # print('"',all_tags[k][i][j].strip(),'",')
      if predictedModels[k][i][j].strip().lower() != 'o':
        result = result+'\''+predictedModels[k][i][j].strip()+'\','
        temLabel = True   
      # print('},')
    if not temLabel:
      result = result + "'O',"
    result = result[0:len(result)-1]
    result = result+'},'
result = result[0:len(result)-1]
print(result)

'paciente':{'O'},'com':{'O'},'sepse':{'B-DiseaseOrSyndrome','B-Disorder'},'pulmonar':{'B-Disorder'},'em':{'I-Disorder'},'d8':{'I-Disorder'},'tazocin':{'I-Disorder'},'(':{'O'},'pciente':{'O'},'não':{'O'},'recebeu':{'O'},'por':{'O'},'2':{'B-QuantitativeConcept'},'dias':{'I-QuantitativeConcept'},'atb':{'O'},')':{'O'},'.':{'O'},'acesso':{'B-MedicalDevice','B-Procedures','B-TherapeuticPreventiveProcedure'},'venoso':{'I-TherapeuticPreventiveProcedure'},'central':{'I-MedicalDevice','I-TherapeuticPreventiveProcedure'},'em':{'O'},'subclavia':{'O'},'d':{'O'},'duplolumen':{'O'},'recebendo':{'O'},'solução':{'O'},'salina':{'O'},'e':{'O'},'glicosada':{'O'},'em':{'O'},'bi':{'O'},'.':{'O'},'Prescrevo':{'O'},'Cetoprofeno':{'O'},'e':{'O'},'descolonização':{'O'},'da':{'O'},'pele':{'O'},'com':{'O'},'Triclosan':{'O'},',':{'O'},'e':{'O'},'manter':{'O'},'Mometasona':{'O'},';':{'O'},'Oriento':{'O'},'buscar':{'O'},'atendimento':{'O'},'com':{'O'},'Dermatologista':{'O'},'caso':{'O'},'não':{'O'},'haja':{'O'},'mel

In [ ]:
# for s,sa in zip(test_sentence_tokenized,tags):
#     for t, a in zip(s,sa):
#         print(t,a)
#     print()

In [ ]:
# Pega o tempo atual menos o tempo do início do processamento.
finalProcessamento = time.time()
tempoTotalProcessamento = formataTempo(finalProcessamento - inicioProcessamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempoTotalProcessamento))


  Tempo processamento:  0:09:30 (h:mm:ss)
